<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using System;
using System.Reactive.Linq;
using System.Windows.Forms;
using WeifenLuo.WinFormsUI.Docking;

// 3DLoadDataクラス
public class 3DLoadData
{
    public 3DLoadData(IObservable<string> dockingEvents)
    {
        // Dockingイベントを購読し、「Hello」をデバッグウィンドウに出力
        dockingEvents.Subscribe(eventInfo =>
        {
            System.Diagnostics.Debug.WriteLine("Hello");
        });
    }
}

// メインフォーム
public class MainForm : Form
{
    private DockPanel dockPanel;
    private IObservable<string> dockingEvents;

    public MainForm()
    {
        dockPanel = new DockPanel();
        this.Controls.Add(dockPanel);
        dockPanel.Dock = DockStyle.Fill;

        // ドッキングイベント用のSubject
        var dockingSubject = new System.Reactive.Subjects.Subject<string>();
        dockingEvents = dockingSubject.AsObservable();

        // 3DLoadDataクラスをインスタンス化し、イベントを購読させる
        var loadData = new 3DLoadData(dockingEvents);

        // フォームを5つ作成し、ドッキング
        for (int i = 1; i <= 5; i++)
        {
            CreateDockableForm(i, dockingSubject);
        }
    }

    private void CreateDockableForm(int number, System.Reactive.Subjects.Subject<string> dockingSubject)
    {
        // フォームの作成
        var form = new Form();
        form.Text = $"Form {number}";
        form.Show(dockPanel, DockState.Document);

        // フォームのDockStateChangedイベントを処理
        form.DockStateChanged += (sender, e) =>
        {
            var dockContent = sender as DockContent;
            if (dockContent != null)
            {
                // ドッキング状態に応じてイベントを発生
                if (dockContent.DockState == DockState.Float)
                {
                    dockingSubject.OnNext($"{form.Text} is floating");
                }
                else if (dockContent.DockState == DockState.Document)
                {
                    dockingSubject.OnNext($"{form.Text} is docked");
                }
            }
        };
    }
}

// プログラムのエントリーポイント
static class Program
{
    [STAThread]
    static void Main()
    {
        Application.EnableVisualStyles();
        Application.SetCompatibleTextRenderingDefault(false);
        Application.Run(new MainForm());
    }
}


In [ ]:
using System;
using System.Windows.Forms;
using WeifenLuo.WinFormsUI.Docking;
using System.Reactive.Linq;

// 5つのフォームにそれぞれ数値を表示する
public class NumberForm : DockContent
{
    public NumberForm(int number)
    {
        Text = $"Number {number}";
        Controls.Add(new Label { Text = number.ToString(), Font = new System.Drawing.Font("Arial", 24F, System.Drawing.FontStyle.Regular, System.Drawing.GraphicsUnit.Point, ((byte)(0))) });
    }
}

// DockPanelの状態変化をイベントで通知するクラス
public class DockPanelStateObserver
{
    public static IObservable<DockState> GetStateObservable(DockPanel dockPanel)
    {
        return Observable.FromEventPattern<DockStateChangedEventHandler, DockStateChangedEventArgs>(
            h => dockPanel.DockStateChanged += h,
            h => dockPanel.DockStateChanged -= h)
            .Select(ep => ep.EventArgs.NewState);
    }
}

public class MainForm : Form
{
    private readonly DockPanel _dockPanel;

    public MainForm()
    {
        // DockPanelのセットアップ
        _dockPanel = new DockPanel();
        Controls.Add(_dockPanel);

        // 5つのフォームを作成してタブにドッキング
        for (int i = 1; i <= 5; i++)
        {
            var numberForm = new NumberForm(i);
            _dockPanel.AddDocument(numberForm);
        }

        // DockPanelの状態変化をObservableで監視
        var stateObservable = DockPanelStateObserver.GetStateObservable(_dockPanel);
        stateObservable.Subscribe(state =>
        {
            // DockPanelの状態に応じて処理を行う
            switch (state)
            {
                case DockState.DockLeft:
                case DockState.DockRight:
                case DockState.DockTop:
                case DockState.DockBottom:
                case DockState.Document:
                    // タブにドッキングされた場合の処理
                    Debug.WriteLine("Hello");
                    break;
                case DockState.Float:
                    // フローティングされた場合の処理
                    Debug.WriteLine("Hello");
                    break;
            }
        });
    }
}

public class 3DLoadData
{
    public 3DLoadData(DockPanel dockPanel)
    {
        // DockPanelの状態変化をObservableで購読
        var stateObservable = DockPanelStateObserver.GetStateObservable(dockPanel);
        stateObservable.Subscribe(state =>
        {
            // DockPanelの状態に応じて処理を行う
            switch (state)
            {
                case DockState.DockLeft:
                case DockState.DockRight:
                case DockState.DockTop:
                case DockState.DockBottom:
                case DockState.Document:
                    // タブにドッキングされた場合の処理
                    Debug.WriteLine("Hello");
                    break;
                case DockState.Float:
                    // フローティングされた場合の処理
                    Debug.WriteLine("Hello");
                    break;
            }
        });
    }
}
